In [1]:
import gc
import torch
from torchvision import transforms

import os
import cv2
import numpy as np
from PIL import Image
from tqdm.notebook import tqdm
from src.utils.image_stuff import image_stuff

In [2]:
def predict(model, data):
    gc.collect()
    torch.cuda.empty_cache()

    # Set model to eval
    model.eval()

    # Get prediction
    with torch.no_grad():
        output = model(data)  
    return output

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [4]:
model = torch.load('model34_2.pth')

model = model.to(device)

In [5]:
dataset_dir_path = "/home/hue/Codes/AIROGS/datasets/5_normalized_224x224"
images_path = [os.path.join(dataset_dir_path,f) for f in sorted(os.listdir(f'{dataset_dir_path}'))][:500]
print(f"Total number of images: {len(images_path)}")

Total number of images: 500


In [6]:
outout_dir_path = "/home/hue/Codes/AIROGS/datasets/CNN_v2_output"
if not os.path.exists(outout_dir_path):
    os.mkdir(outout_dir_path)

In [7]:
transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

In [8]:
def read_images(images_path):
    for path in images_path:
        yield [Image.open(path), os.path.basename(path)]

In [9]:
images = read_images(images_path)

In [10]:
# https://stackoverflow.com/questions/434287/what-is-the-most-pythonic-way-to-iterate-over-a-list-in-chunks/434328
from iteration_utilities import grouper


for group in tqdm(grouper(images, 100), total=len(images_path)//100):
# https://stackoverflow.com/questions/67814465/convert-list-of-tensors-into-tensor-pytorch
    data = torch.stack([transform(image[0]) for image in group]).to(device)
    output = predict(model, data)
    
    for idx in range(data.shape[0]):
        bbox_image = image_stuff.draw_bbox(np.array(group[idx][0]),
                                           output.data.int()[idx],
                                           "deep_learning",
                                           box_color=(0, 225, 0))
        
        cv2.imwrite(os.path.join(outout_dir_path,f"{group[idx][1][:-4]}_bbox.jpg"),
                    cv2.cvtColor(bbox_image, cv2.COLOR_RGB2BGR))
        with open(os.path.join(outout_dir_path, f"{group[idx][1][:-4]}.txt"), "w") as f:
            for coordinate in output.data.int()[idx]:
                f.write(f"{int(coordinate)}\n") 

#         image_stuff.plt_display_image(bbox_image, figsize=(10,10))
    

/home/hue/miniconda3/lib/python3.9/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
